In [ ]:
import warnings
warnings.filterwarnings('ignore')
import glob
import os

import cv2
from keras.preprocessing.image import load_img, save_img
from keras.utils import get_file
import numpy as np

from utils import config

### Download Zappos50K dataset

In [ ]:
get_file(f'{config.ZAPPOS_DATASET_NAME}.zip', config.ZAPPOS_DATASET_URL, extract=True, 
         cache_dir='.', cache_subdir='')

### Define Functions

In [ ]:
def create_source_imgs(target_dir, source_dir):
    pathname = f'{target_dir}/*.jpg'
    for filepath in glob.glob(pathname):
        img_target = load_img(filepath, target_size=(config.IMG_HEIGHT, config.IMG_WIDTH))
        img_target = np.array(img_target)
        img_source = detect_edges(img_target)

        filename = os.path.basename(filepath)
        img_source_filepath = os.path.join(source_dir, filename)
        save_img(img_source_filepath, img_source)

def create_target_images():
    pathname = f'{config.ZAPPOS_DATASET_SNEAKERS_DIR}/*/*.jpg'
    for filepath in glob.glob(pathname):
        filename = os.path.basename(filepath)
        img_target = load_img(filepath, target_size=(config.IMG_HEIGHT, config.IMG_WIDTH))
        img_target = np.array(img_target)
        img_target_filepath = os.path.join(config.TRAINING_TARGET_DIR, filename)
        save_img(img_target_filepath, img_target)
            
def detect_edges(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_gray = cv2.bilateralFilter(img_gray, 5, 50, 50)
    img_gray_edges = cv2.Canny(img_gray, 45, 100)
    img_gray_edges = cv2.bitwise_not(img_gray_edges) # invert black/white
    img_edges = cv2.cvtColor(img_gray_edges, cv2.COLOR_GRAY2RGB)
    
    return img_edges

def resize_imgs_in_directory(img_dir):
    for filepath in glob.glob(f'{img_dir}/*.jpg'):
        img = load_img(filepath, target_size=(config.IMG_HEIGHT, config.IMG_WIDTH))
        img = np.array(img)
        save_img(filepath, img)

##  Training Set

In [ ]:
create_target_images()

In [ ]:
create_source_imgs(config.TRAINING_TARGET_DIR, config.TRAINING_SOURCE_DIR)

## Validation Set

In [ ]:
resize_imgs_in_directory(config.VALIDATION_TARGET_DIR)

In [ ]:
create_source_imgs(config.VALIDATION_TARGET_DIR, config.VALIDATION_SOURCE_DIR)